In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
#Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv("cities (1).csv")
cities_df.head()

,city,country,longitude,latitude,Temperature F,Humidity %,Cloudiness,Wind Speed mph
0,Ushuaia,AR,-68.30,-54.80,42.80,81,10000,36.91
1,Batouri,CM,14.37,4.43,70.95,61,10000,1.74
2,Salinas,US,-121.66,36.68,46.40,81,10000,2.57
3,Port Antonio,JM,-76.46,18.18,78.80,78,10000,6.93
4,Kavieng,PG,150.80,-2.57,85.28,70,10000,6.76


In [4]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
cities = cities_df[["latitude", "longitude"]]
humidity = cities_df["Humidity %"]

#Add Heatmap layer to map.
city_fig = gmaps.figure()
# Create heat layer
heat_map = gmaps.heatmap_layer(cities, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
city_fig.add_layer(heat_map)

city_fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

cities_df.count()

city              554
country           548
longitude         554
latitude          554
Temperature F     554
Humidity %        554
Cloudiness        554
Wind Speed mph    554
dtype: int64

In [6]:
#Store into variable named hotel_df.
hotel_df = cities_df.dropna(how='any')
hotel_df.count()

city              548
country           548
longitude         548
latitude          548
Temperature F     548
Humidity %        548
Cloudiness        548
Wind Speed mph    548
dtype: int64

,city,country,longitude,latitude,Temperature F,Humidity %,Cloudiness,Wind Speed mph
0,Ushuaia,AR,-68.30,-54.80,42.80,81,10000,36.91
1,Batouri,CM,14.37,4.43,70.95,61,10000,1.74
2,Salinas,US,-121.66,36.68,46.40,81,10000,2.57
3,Port Antonio,JM,-76.46,18.18,78.80,78,10000,6.93
4,Kavieng,PG,150.80,-2.57,85.28,70,10000,6.76


In [95]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
i = 0
hotels = []
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
for index, row in hotel_df.iterrows():
    
    longitude = str(row["longitude"])
    latitude = str(row["latitude"])
    coordinate = latitude + "," + longitude
    cities_param = coordinate
    site_type = "lodging"
    radius = 5000

    parameters = {"location" : cities_param,
                    "radius" : radius,
                    "type" : site_type,
                    "key" : g_key}
    response = requests.get(url, params=parameters)
    #print(url, response.url)
    #print(response) #to check if responses are properly requested
    hotel_data = response.json()
   
    #pprint(hotel_data) #double check to ensure requests were recieved
    print("Processing Record ")
    try:
        hotels.append(hotel_data['results'][1]['name'])
        #print(hotels)
    except:
        hotels.append("N/A")
#print(hotels) # ensures hotels append properly

Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing Record 
Processing R

In [97]:
hotel_df["Hotel Names"] = hotels
hotel_df.head()

/Users/tonypalazzolo/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,city,country,longitude,latitude,Temperature F,Humidity %,Cloudiness,Wind Speed mph,Hotel Names
0,Ushuaia,AR,-68.30,-54.80,42.80,81,10000,36.91,Las Lengas Hotel
1,Batouri,CM,14.37,4.43,70.95,61,10000,1.74,Hotel Belle etoile
2,Salinas,US,-121.66,36.68,46.40,81,10000,2.57,Laurel Inn & Conference Center
3,Port Antonio,JM,-76.46,18.18,78.80,78,10000,6.93,Finjam Cottage
4,Kavieng,PG,150.80,-2.57,85.28,70,10000,6.76,KVR TOWN OFFICE


In [100]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [102]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

marker_locations = locations

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))